In [1]:
from custom_envs.dm_control_suite import get_default_config, load, get_domain_randomizer
from custom_envs.manipulation import get_default_config, load, get_domain_randomizer
import jax
import jax.numpy as jnp
import os
from learning.module.wrapper.adv_wrapper import wrap_for_adv_training
from learning.module.wrapper.dr_wrapper import wrap_for_dr_training
import functools
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]='false'
import mediapy as media


In [2]:
env_name='LeapCubeReorient'
cfg = get_default_config(env_name)
env = load(env_name, cfg)
# randomizer = get_domain_randomizer(env_name)
# randomizer = functools.partial(randomizer, dr_range=env.dr_range)

# eval_env = wrap_for_adv_training(
#       env,
#       episode_length=1000,
#       action_repeat=2,
#       randomization_fn=randomizer,
# )
# eval_env2 = wrap_for_dr_training(
#       env,
#       episode_length=1000,
#       action_repeat=2,
#       randomization_fn=randomizer,
#       n_envs=1,
#       n_nominals=1,
# )


In [3]:
jit_step = jax.jit(env.step)
jit_reset = jax.jit(env.reset)


In [24]:
init_state = jit_reset(jax.random.PRNGKey(2))
state = init_state


In [26]:
seconds = 3
rollout=[state]
for i in range(int(seconds/env.sim_dt)):
    state = jit_step(state, jnp.ones(env.action_size))
    rollout.append(state)
frames = env.render(rollout)
media.show_video(frames, fps=1.0 / env.sim_dt)


100%|██████████| 301/301 [00:01<00:00, 286.54it/s]


This browser does not support the video tag.


In [27]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i), 
        state.data.xpos[i]
    ) 
    for i in range(env.mj_model.nbody)
]


[('world', Array([0., 0., 0.], dtype=float32)),
 ('leap_mount', Array([0., 0., 0.], dtype=float32)),
 ('palm', Array([ 0.124  , -0.0305 , -0.06045], dtype=float32)),
 ('if_bs', Array([ 0.12376653, -0.0535    , -0.04048414], dtype=float32)),
 ('if_px', Array([ 0.12364316, -0.039     , -0.00047871], dtype=float32)),
 ('if_md', Array([ 0.1051522 , -0.03316167,  0.01443733], dtype=float32)),
 ('if_ds', Array([ 0.07426076, -0.04272743, -0.00160841], dtype=float32)),
 ('mf_bs', Array([ 0.12367246, -0.0081    , -0.04045018], dtype=float32)),
 ('mf_px', Array([ 0.15188977,  0.0063999 , -0.0120911 ], dtype=float32)),
 ('mf_md', Array([0.14936988, 0.01228053, 0.01152158], dtype=float32)),
 ('mf_ds', Array([0.11823228, 0.01577415, 0.02945184], dtype=float32)),
 ('rf_bs', Array([ 0.12368187,  0.0373    , -0.04045358], dtype=float32)),
 ('rf_px', Array([ 0.14518757,  0.05179995, -0.00671996], dtype=float32)),
 ('rf_md', Array([0.13768217, 0.05765513, 0.01581613], dtype=float32)),
 ('rf_ds', Array([

In [7]:
env.mjx_model.body_ipos


Array([[ 0.        ,  0.        ,  0.        ],
       [ 0.04366667,  0.031     ,  0.00666667],
       [-0.049542  , -0.042914  , -0.010227  ],
       [-0.022516  ,  0.033882  ,  0.016359  ],
       [ 0.0075    , -0.0002    , -0.011     ],
       [ 0.0054215 , -0.029148  ,  0.015     ],
       [-0.0008794 , -0.027019  ,  0.014594  ],
       [-0.022516  ,  0.033882  ,  0.016359  ],
       [ 0.0075    , -0.0002    , -0.011     ],
       [ 0.0054215 , -0.029148  ,  0.015     ],
       [-0.0008794 , -0.027019  ,  0.014594  ],
       [-0.022516  ,  0.033882  ,  0.016359  ],
       [ 0.0075    , -0.0002    , -0.011     ],
       [ 0.0054215 , -0.029148  ,  0.015     ],
       [-0.0008794 , -0.027019  ,  0.014594  ],
       [ 0.0075    , -0.0002    , -0.011     ],
       [ 0.        ,  0.        , -0.0070806 ],
       [-0.0020593 ,  0.015912  , -0.013733  ],
       [ 0.00096191, -0.024203  , -0.014419  ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.   

In [8]:
state.data.geom_xpos


Array([[ 0.00000000e+00,  0.00000000e+00, -2.50000000e-01],
       [ 1.50264613e-02,  1.31287202e-02, -1.59481093e-02],
       [ 4.05000001e-02,  1.99999996e-02,  8.99999961e-03],
       [ 5.00000007e-02,  5.29999994e-02,  2.00000009e-03],
       [ 8.34393650e-02,  7.18123280e-03, -3.38105187e-02],
       [ 1.19270466e-01, -3.84999998e-02, -4.70473319e-02],
       [ 1.19270466e-01,  6.50000013e-03, -4.70473319e-02],
       [ 1.19270466e-01,  5.15000001e-02, -4.70473319e-02],
       [ 6.58277795e-02,  4.74999994e-02, -2.72198617e-02],
       [ 9.68016982e-02,  4.49999981e-03, -4.74402979e-02],
       [ 1.02052152e-01,  4.49999981e-03, -2.70091258e-02],
       [ 8.27580318e-02,  2.49999948e-03, -3.33326533e-02],
       [ 5.45409396e-02,  2.24999990e-02, -2.31446624e-02],
       [ 3.38617340e-02, -2.15000007e-02, -2.15258040e-02],
       [ 4.72395346e-02, -2.74999999e-02, -4.33669165e-02],
       [ 1.31894663e-01, -3.79217565e-02, -6.70518354e-03],
       [ 1.18224457e-01, -3.87999974e-02

In [9]:
env.mjx_model.geom_friction


Array([[1.e+00, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [7.e-01, 5.e-02, 2.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.e-01, 5.e-03, 1.e-04],
       [2.

In [10]:
env.mjx_model.dof_frictionloss.shape


(22,)

In [11]:
env.mjx_model.nv


22

In [12]:
env.mjx_model.nu


16

In [13]:
env.mjx_model.qpos0.shape


(23,)

In [14]:
from mujoco import mj_id2name, mj_name2id
import mujoco
[mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i) for i in range(env.mj_model.nbody)]


['world',
 'leap_mount',
 'palm',
 'if_bs',
 'if_px',
 'if_md',
 'if_ds',
 'mf_bs',
 'mf_px',
 'mf_md',
 'mf_ds',
 'rf_bs',
 'rf_px',
 'rf_md',
 'rf_ds',
 'th_mp',
 'th_bs',
 'th_px',
 'th_ds',
 'cube',
 'goal']

In [15]:

[
    (
        mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_JOINT, i),
        env.mj_model.dof_frictionloss[i]
    )
    for i in range(env.mj_model.njnt)]


[('if_mcp', np.float64(0.02)),
 ('if_rot', np.float64(0.02)),
 ('if_pip', np.float64(0.02)),
 ('if_dip', np.float64(0.02)),
 ('mf_mcp', np.float64(0.02)),
 ('mf_rot', np.float64(0.02)),
 ('mf_pip', np.float64(0.02)),
 ('mf_dip', np.float64(0.02)),
 ('rf_mcp', np.float64(0.02)),
 ('rf_rot', np.float64(0.02)),
 ('rf_pip', np.float64(0.02)),
 ('rf_dip', np.float64(0.02)),
 ('th_cmc', np.float64(0.02)),
 ('th_axl', np.float64(0.02)),
 ('th_mcp', np.float64(0.02)),
 ('th_ipl', np.float64(0.02)),
 ('cube_freejoint', np.float64(0.0))]

In [16]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_GEOM, i), 
        env.mj_model.geom_friction[i]
    ) 
    for i in range(env.mj_model.ngeom)
]


[('floor', array([1.e+00, 5.e-03, 1.e-04])),
 ('leap_mount_visual', array([2.e-01, 5.e-03, 1.e-04])),
 ('leap_mount_collision_0', array([2.e-01, 5.e-03, 1.e-04])),
 ('leap_mount_collision_1', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_visual', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_1', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_2', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_3', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_4', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_5', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_6', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_7', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_8', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_9', array([2.e-01, 5.e-03, 1.e-04])),
 ('palm_collision_10', array([2.e-01, 5.e-03, 1.e-04])),
 ('if_bs_visual', array([2.e-01, 5.e-03, 1.e-04])),
 ('if_bs_collision_1', array([2.e-01, 5.e-03, 1.e-04])),
 ('if_px_visual', array([2.e-01, 5.e-03, 1.e-04])),
 

In [17]:
env.mj_model.geom_friction[0]



array([1.e+00, 5.e-03, 1.e-04])

In [18]:
[
    (
        mujoco.mj_id2name(env.mj_model, mujoco.mjtObj.mjOBJ_BODY, i), 
        env.mj_model.body_mass[i]
    ) 
    for i in range(env.mj_model.nbody)
]


[('world', np.float64(0.0)),
 ('leap_mount', np.float64(0.003000000000000001)),
 ('palm', np.float64(0.237)),
 ('if_bs', np.float64(0.044)),
 ('if_px', np.float64(0.032)),
 ('if_md', np.float64(0.037)),
 ('if_ds', np.float64(0.016)),
 ('mf_bs', np.float64(0.044)),
 ('mf_px', np.float64(0.032)),
 ('mf_md', np.float64(0.037)),
 ('mf_ds', np.float64(0.016)),
 ('rf_bs', np.float64(0.044)),
 ('rf_px', np.float64(0.032)),
 ('rf_md', np.float64(0.037)),
 ('rf_ds', np.float64(0.016)),
 ('th_mp', np.float64(0.032)),
 ('th_bs', np.float64(0.003)),
 ('th_px', np.float64(0.038)),
 ('th_ds', np.float64(0.049)),
 ('cube', np.float64(0.108)),
 ('goal', np.float64(0.108))]

In [19]:

init_state=  env.reset(jax.random.PRNGKey(0))
state=init_state



In [20]:
from custom_envs import mjx_env


state.data
mjx_env.get_sensor_data(
        env.mj_model, state.data, "torso_subtreelinvel"
    )


KeyError: "Invalid name 'torso_subtreelinvel'. Valid names: ['cube_angacc', 'cube_angvel', 'cube_goal_orientation', 'cube_goal_upvector', 'cube_linvel', 'cube_orientation', 'cube_position', 'cube_upvector', 'if_tip_position', 'mf_tip_position', 'palm_position', 'rf_tip_position', 'th_tip_position']"

In [ ]:
state = env.step(state, jnp.ones(env.action_size))
state.data.xpos



Array([[ 0.        ,  0.        ,  0.        ],
       [ 0.00229061, -0.09994751,  0.05      ],
       [ 0.04339438, -0.18141547,  0.05      ],
       [ 0.13138694, -0.20391439,  0.05      ],
       [ 0.22999373, -0.20243815,  0.05      ],
       [ 0.3164212 , -0.24361372,  0.05      ],
       [ 0.3989326 , -0.29794952,  0.05      ],
       [-1.3853827 ,  0.91602135,  0.05      ]], dtype=float32)

In [ ]:
from learning.module.wrapper.evaluator import AdvEvaluator


evaluator = AdvEvaluator(
        env,
        lambda x : x,
        num_eval_envs=128,
        episode_length=1000,
        action_repeat=2,
        key=jax.random.PRNGKey(0),
    )
dynamics_params = jnp.zeros((128,14))
metrics = evaluator.run_evaluation(dynamics_params, dynamics_params, {})
